In [291]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pathlib import Path
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans
import seaborn as sns
import src.stimuli_timeline as st
import src.plotting as plott
plt.style.use('default')
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Stimuli Traces


##### Define file paths

In [292]:
# Experiment config
# Defines run-specific parameters: fish/experiment IDs, FPS, planes, stimulus labels/colors, pre/post windows, and filename prefixes.
fish_id = "L472_f06"  # 👈 just change this
experiment_name = 'Exp_2_rocking_1'  # 👈 just change this
fish = f"{fish_id}_{experiment_name}"
#stimuli path
#stimuli_path = Path(r"\\nasdcsr.unil.ch\RECHERCHE\FAC\FBM\CIG\jlarsch\default\D2c\Alejandro\2p\flickering\stimuli")
stimuli_main_path = Path(r"\\nasdcsr.unil.ch\RECHERCHE\FAC\FBM\CIG\jlarsch\default\D2c\Alejandro\2p")
stimuli_path =  stimuli_main_path / experiment_name  / "stimuli"
#experiment path
main_path = Path(r"C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p")
# --- FIND THE CORRECT LOG FILE ---
metadata_dir = main_path / fish / "01_raw" / "2p" / "metadata"
# deltaF/F path
dfof_dir= main_path / fish / "03_analysis" / "functional" / "suite2P" / "merged_dFoF"
# plots saving path
plots_path = main_path / fish / "03_analysis" / "functional" / "plots" / "merged_dFoF"

In [293]:
# Load deltaF/F data
c = [p for p in dfof_dir.glob("*.npy") if "dfof_merged" in p.name.lower()]
dfof = np.load(c[0])
# print("✅ Loaded:", dfof)
print("Shape:", dfof.shape)

Shape: (3340, 1363)


In [294]:
# Inputs
fps_2p = 2
selected_blocks = [f'B{n}' for n in range(1, 3)]
# number of frames from dfof
frames_per_block = dfof.shape[0] / len(selected_blocks)
print(frames_per_block)

1670.0


##### Load Stimuli Durations

In [295]:
stimuli_durations = {}

for stim_file in stimuli_path.glob("*trajectory.*"):
    filename = stim_file.stem  # e.g., 'FL2_trajectory'
    stim_name = filename.replace("_trajectory", "")  # e.g., 'FL2'

    if any(s in filename for s in ("B", "RR", "RL")):
        stimuli_durations[stim_name] = st.get_stimulus_timing(stim_file)
    else:
        stimuli_durations[stim_name] = st.get_radius_timing(stim_file)


In [296]:
# it converts each stimulus record to include motion_sec, zeroes static_after_sec, and normalizes the end frame fields to total_frames (falling back to motion_end_frame) without touching the original input.

def transform_stimuli_duration(d):
    out = {}
    for k, v in d.items():
        total_sec = v.get('total_sec', 0)
        static_before = v.get('static_before_sec', 0)
        total_frames = v.get('total_frames', v.get('motion_end_frame'))

        new_v = v.copy()
        new_v['motion_sec'] = round(total_sec - static_before, 3)
        new_v['static_after_sec'] = 0
        if total_frames is not None:
            new_v['motion_end_frame'] = total_frames
            new_v['end_frame'] = total_frames
        out[k] = new_v
    return out

new_data = transform_stimuli_duration(stimuli_durations)

#print(new_data)

##### Compute experiment stimuli traces

In [297]:
# build stimulus traces aligned to 2P time
# Reads the block-log CSV, time-shifts selected blocks to a common 2P timeline, and generates binary time series for each requested trace  using per-stimulus durations.

# find all CSVs that end with "block_log.csv"
stimuli_durations=new_data #updates stimuli_durations to the transformed new_data from earlier
block_logs = list(metadata_dir.glob("*block_log.csv"))

if not block_logs:
    raise FileNotFoundError(f"No '*block_log.csv' file found in {metadata_dir}")
elif len(block_logs) > 1:
    print("⚠️ Multiple block log files found, using the most recent one:")
    for f in block_logs:
        print(" -", f.name)
    # sort by modification time and pick the newest
    block_logs.sort(key=lambda p: p.stat().st_mtime, reverse=True)

experiment_log_path = block_logs[0]
print("✅ Using block log:", experiment_log_path)

# --- EXAMPLE USAGE ---

duration_2p_block_sec = frames_per_block / fps_2p

traces_dict, exp_log = st.make_stimulus_traces(
    experiment_log_path,
    stimuli_durations,
    selected_blocks,
    duration_2p_block_sec
)


✅ Using block log: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L472_f06_Exp_2_rocking_1\01_raw\2p\metadata\2025-09-03-1635_L472_f06_block_log.csv


In [298]:
# select colors and linestyles for stimuli
# --- Set A (default) ----------------------------------------------------------
colors_A = {
    'LLB': sns.color_palette('Blues', 8)[7],
    'RLB': sns.color_palette('Greens', as_cmap=True)(0.6),
    'FLB': sns.color_palette('Blues', 8)[7],
    'FRB': sns.color_palette('Greens', as_cmap=True)(0.6),
    'FL1': sns.color_palette('PuRd', as_cmap=True)(0.9),
    'FL2': sns.color_palette('PuRd', as_cmap=True)(0.6),
    'FL3': sns.color_palette('dark:#404040', 10)[7],
    'FR1': sns.color_palette('PuRd', as_cmap=True)(0.9),
    'FR2': sns.color_palette('PuRd', as_cmap=True)(0.6),
    'FR3': sns.color_palette('dark:#404040', 10)[7],
}
linestyles_A = {
    'FLB': '--',
    'FRB': '--',
    'FL1': '--',
    'FL2': '--',
    'FL3': '--',
}

# --- Set B (rocking experiment) ----------------------------------------------
colors_B = {
    'LB':   sns.color_palette('Blues', 8)[7],
    'RR1':  sns.color_palette('Greens', as_cmap=True)(0.6),
    'RB':   sns.color_palette('Blues', 8)[7],
    'RL1':  sns.color_palette('Greens', as_cmap=True)(0.6),
    'RR2':  sns.color_palette('PuRd', as_cmap=True)(0.9),
    'RR3':  sns.color_palette('PuRd', as_cmap=True)(0.6),
    'RLR1': sns.color_palette('dark:#404040', 10)[7],
    'RL2':  sns.color_palette('PuRd', as_cmap=True)(0.9),   # fixed case from 'Rl2' → 'RL2'
    'RL3':  sns.color_palette('PuRd', as_cmap=True)(0.6),
    'RLR2': sns.color_palette('dark:#404040', 10)[7],
}
linestyles_B = {
    'LB': '--',
    'RL1': '--',
    'RL2': '--',
    'RL3': '--',
    'RLR2': '--',
}

# --- Select set based on experiment name --------------------------------------
if experiment_name == 'Exp_2_rocking_1':
    stimuli_colors = colors_B
    stimuli_linestyles = linestyles_B
else:
    stimuli_colors = colors_A
    stimuli_linestyles = linestyles_A

print("Using colors for:", experiment_name)
print(stimuli_linestyles)

Using colors for: Exp_2_rocking_1
{'LB': '--', 'RL1': '--', 'RL2': '--', 'RL3': '--', 'RLR2': '--'}





Plot

In [299]:
#plot stimuli as a binary trace
fps_stim=60
fig, ax = plt.subplots(figsize=(15, 2))
plott.add_stimuli_markers(ax, exp_log, stimuli_durations, stimuli_colors,stimuli_linestyles=stimuli_linestyles)
plt.title('Movement Periods')
plt.xlabel('Time (sec)')
plt.ylabel('Movement (Binary)')
plt.grid(True)
plt.ylim(0, 1.1)
plt.xlim(0, len(traces_dict['movement']) / fps_stim)
plt.tight_layout()
plt.show()


In [308]:
# assume fps_stim is defined (e.g., 60) and ax is your matplotlib axes
for stim_name, timing in stimuli_durations.items():
    start_f = timing.get('start_frame', timing.get('motion_start_frame'))
    end_f   = timing.get('end_frame',   timing.get('motion_end_frame'))
    if start_f is None or end_f is None:
        print(f"Skipping {stim_name}: no start/end frame keys")
        continue

    start = start_f / fps_stim
    end   = end_f   / fps_stim
    ax.axvspan(start, end, alpha=0.3)  # avoid hardcoding a color unless you want to


### Delta F - fluorescent traces

### Plots

##### All Planes raster plots with line for stimulus onset

In [301]:
# plot full dfof raster with stimuli markers no sorting
deltaF_F = dfof  # (frames x neurons)

fig, ax = plt.subplots(figsize=(12, 4))

# call your function — use vmax (or vmin/vmax) depending on your signature
im = plott.raster_with_stimuli(
    ax=ax,
    deltaF_F=deltaF_F,
    fps=fps_2p,
    fish_id=fish_id,
    max=0.3,        # if your function still uses min/max, use min=0, max=0.4 instead
)

# add stimuli markers & legend on the SAME axes
legend_handles = plott.add_stimuli_markers(ax, exp_log, stimuli_durations, stimuli_colors,stimuli_linestyles=stimuli_linestyles)
ax.legend(handles=legend_handles, title="Stimuli",
          bbox_to_anchor=(1.2, 1), loc='upper left', borderaxespad=0, frameon=False)

ax.set_xlabel("Time (s)")

# add colorbar to the SAME figure
fig.colorbar(im, ax=ax, label="ΔF/F")

fig.tight_layout()
plt.show()



##### Plane 3 plot across clustering methods

#### Grouped rasterplots

In [302]:
# select parameters including sorting mode, stimuli order, and windows
# sort deltaF_F base on max_intensity, correlation, kmeans, "pca", "hier",  or "unsorted"
# --- choose stimulus-aligned windows ---

# --- choose neuron sorting ---
window_pre = 5  # seconds before stimulus
window_post = 17  # seconds after stimulus


# <-- change to: max_intensity, correlation, kmeans, "pca", "hier",  or "unsorted"
sort_mode = "correlation"

# define stimuli order based on experiment
stimuli_ordered_A = ['LLB', 'FLB', 'RLB' , 'FRB','FL1', 'FL2', 'FL3', 'FR1', 'FR2', 'FR3']  # Order of
stimuli_ordered_B = ['LB','RB','RR1','RR2','RR3','RL1','RL2','RL3','RLR1','RLR2' ]

if experiment_name == 'Exp_2_rocking_1':
    stimuli_ordered = stimuli_ordered_B
else:
    stimuli_ordered = stimuli_ordered_A


# (a) Max intensity (peak ΔF/F per neuron)
max_per_neuron   = np.nanmax(deltaF_F, axis=0)         # (neurons,)
maxint_sorted_idx = np.argsort(-max_per_neuron)

# (b) PCA (PC1 desc) over neurons
scores    = PCA(n_components=3).fit_transform(deltaF_F.T )  # (neurons, 3)
PCA_order = np.argsort(-scores[:, 0])

# (c) KMeans over neurons
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(deltaF_F.T )
kmeans_sorted_idx = np.argsort(kmeans.labels_)  # group by label

# (d) Hierarchical (Ward) over neurons
Z_hier = linkage(deltaF_F.T , method='ward')
clusters = fcluster(Z_hier, t=5, criterion='maxclust')
hier_sorted_idx = np.argsort(clusters)

# (e) Correlation sort on the **averaged chunks** (neurons x time)
dist = pdist(deltaF_F.T, metric='correlation')
Z_corr = linkage(dist, method='average')
corravg_sorted_idx = leaves_list(Z_corr)


n_neurons = dfof.shape[1]

def _validate_order(idx, name):
    if idx is None:
        raise ValueError(f"{name} is not defined but sort_mode='{name}' was selected.")
    if len(idx) != n_neurons:
        raise ValueError(f"{name} length ({len(idx)}) != number of neurons ({n_neurons}).")
    return idx

if sort_mode == "kmeans":
    neuron_order = _validate_order(kmeans_sorted_idx, "kmeans_sorted_idx")
elif sort_mode == "pca":
    neuron_order = _validate_order(PCA_order, "PCA_order")
elif sort_mode == "hier":
    neuron_order = _validate_order(hier_sorted_idx, "hier_sorted_idx")
elif sort_mode == "max_intensity":
    neuron_order = _validate_order(maxint_sorted_idx, "maxint_sorted_idx")
elif sort_mode == "correlation":
    neuron_order = _validate_order(corravg_sorted_idx, "corravg_sorted_idx")
elif sort_mode == "unsorted":
    neuron_order = None
else:
    raise ValueError("sort_mode must be one of: 'kmeans', 'pca', 'hier', 'max_intensity', 'unsorted'")
print(stimuli_ordered)

C:\Users\suribear\AppData\Local\anaconda3\envs\neuro-group\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


['LB', 'RB', 'RR1', 'RR2', 'RR3', 'RL1', 'RL2', 'RL3', 'RLR1', 'RLR2']


##### Grouped chuncks ordered per stimulus type

In [303]:
# gruped chunks ordered by stimulus type, sorted by selected method

# 1) Build chunks
chunked_data, trial_starts, move_starts, move_colors, stim_labels = st.extract_stimulus_chunks(
    deltaF_F=dfof,
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    fps=fps_2p,
    stimuli_ordered=stimuli_ordered,
    window_pre=window_pre,
    window_post=window_post,
    average_across_repeats=False,
)

if chunked_data is None or chunked_data.size == 0:
    raise ValueError("No stimulus chunks were extracted. Check exp_log, stimuli_ordered, and windows.")

# 2) Plot raster
fig, ax = plt.subplots(figsize=(12, 6))
im = plott.raster_with_stimuli(
    ax=ax,
    deltaF_F=chunked_data.T,   # (time, neurons)
    fps=fps_2p,
    fish_id=fish_id,
    neuron_order=neuron_order, # provided by your sort selector
    title_suffix=f"ordered by stimulus type | sort={sort_mode}",
    max=0.3
)

# 3) Movement start lines — use style based on each chunk's stimulus label
move_styles = [stimuli_linestyles.get(name, '-') for name in stim_labels]
for pos, color, ls in zip(move_starts, move_colors, move_styles):
    ax.axvline(pos / fps_2p, color=color, linestyle=ls, alpha=0.9, linewidth=1.0)

ax.set_xlabel("Chunks aligned to stimulus onset (s)")

# 4) Legend that matches color + linestyle
legend_handles = []
for stim_name, color in stimuli_colors.items():
    ls = stimuli_linestyles.get(stim_name, '-')
    (line,) = ax.plot([], [], color=color, linestyle=ls, label=stim_name, linewidth=2)
    legend_handles.append(line)

ax.legend(
    handles=legend_handles,
    title="Movement onset\nacross stimuli",
    bbox_to_anchor=(1.15, 1),
    loc="upper left",
    borderaxespad=0,
    frameon=False
)

# 5) Colorbar + layout
fig.colorbar(im, ax=ax, label="ΔF/F")
fig.tight_layout()
plt.show()



In [304]:
# Purpose.
# Generate experiment-level ΔF/F rasters aligned to stimulus onsets, averaged across repeats, and saved once per sorting strategy (unsorted, max intensity, PCA, KMeans, hierarchical, and correlation on averaged responses).

import os
os.environ["OMP_NUM_THREADS"] = "2"  # or "1"

# ----- pick a short prefix like "L433_f03" from "L433_f03_Exp_1_flickering"
prefix = "_".join(fish.split("_")[:2])

# ----- ensure output dir exists
plots_path.mkdir(parents=True, exist_ok=True)

# ===== 1) Build chunks (averaged across repeats) once =====
chunked_data, trial_starts, move_starts, move_colors, stim_labels = st.extract_stimulus_chunks(
    deltaF_F=dfof,                 # (frames, neurons)
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    fps=fps_2p,
    stimuli_ordered=stimuli_ordered,
    window_pre=window_pre,
    window_post=window_post,
    average_across_repeats=False
)
if chunked_data is None or chunked_data.size == 0:
    raise ValueError("No stimulus chunks were extracted. Check exp_log, stimuli_ordered, and windows.")
# chunked_data: (neurons, time)

# ===== 2) Precompute neuron orders for each sort mode (on full dfof unless noted) =====
deltaF_F = dfof  # (frames, neurons)

# (a) Max intensity (peak ΔF/F per neuron)
max_per_neuron   = np.nanmax(chunked_data.T, axis=0)         # (neurons,)
maxint_sorted_idx = np.argsort(-max_per_neuron)

# (b) PCA (PC1 desc) over neurons
scores    = PCA(n_components=3).fit_transform(chunked_data)  # (neurons, 3)
PCA_order = np.argsort(-scores[:, 0])

# (c) KMeans over neurons
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(chunked_data)
kmeans_sorted_idx = np.argsort(kmeans.labels_)  # group by label

# (d) Hierarchical (Ward) over neurons
Z_hier = linkage(chunked_data, method='ward')
clusters = fcluster(Z_hier, t=5, criterion='maxclust')
hier_sorted_idx = np.argsort(clusters)

# (e) Correlation sort on the **averaged chunks** (neurons x time)
dist = pdist(chunked_data, metric='correlation')
Z_corr = linkage(dist, method='average')
corravg_sorted_idx = leaves_list(Z_corr)

# Map from mode -> (index array or None, label suffix)
sorters = {
    "unsorted":        (None,                   "unsorted"),
    "max_intensity":   (maxint_sorted_idx,      "max_intensity"),
    "pca":             (PCA_order,              "pca"),
    "kmeans":          (kmeans_sorted_idx,      "kmeans"),
    "hier":            (hier_sorted_idx,        "hier"),
    "corravg":         (corravg_sorted_idx,     "corravg"),  # correlation on averaged chunks
}


def _validate_order(idx, n_neurons, name):
    if idx is None:
        return None
    if len(idx) != n_neurons:
        raise ValueError(f"{name} length ({len(idx)}) != number of neurons ({n_neurons}).")
    return idx

# ===== 3) Plot once per sort mode (use chunked_data for raster) =====
for mode, (idx, tag) in sorters.items():
    neuron_order = _validate_order(idx, chunked_data.shape[0], name=mode)

    fig, ax = plt.subplots(figsize=(12, 6))
    im = plott.raster_with_stimuli(
        ax=ax,
        deltaF_F=chunked_data.T,            # (time, neurons)
        fps=fps_2p,
        fish_id=fish_id,
        neuron_order=neuron_order,          # None = original order
        title_suffix=f"ordered by stimulus type | sort={tag}",
        min=0.009,
        max=0.3
    )

    # Movement onset lines with per-stim color + linestyle
    for pos, stim_name in zip(move_starts, stim_labels):
        color = stimuli_colors.get(stim_name, 'black')
        ls    = stimuli_linestyles.get(stim_name, '-')  # default solid
        ax.axvline(pos / fps_2p, color=color, linestyle=ls, alpha=0.9, linewidth=1.8)

    # Legend (color + linestyle)
    legend_handles = []
    for stim_name in stimuli_ordered:
        if stim_name in stimuli_colors:
            color = stimuli_colors[stim_name]
            ls    = stimuli_linestyles.get(stim_name, '-')
            (line,) = ax.plot([], [], color=color, linestyle=ls, label=stim_name, linewidth=1.5)
            legend_handles.append(line)

    leg = ax.legend(
        handles=legend_handles, title="Movement onset\nacross stimuli",
        bbox_to_anchor=(1.2, 1), loc='upper left',
        borderaxespad=0, frameon=False
    )

    ax.set_xlabel("Chunks aligned to stimulus onset (s)")
    fig.colorbar(im, ax=ax, label="ΔF/F")
    plt.subplots_adjust(right=0.85)

    # Save
    out_png = plots_path / f"{prefix}_grouped_dfof_sorted_by_{tag}.png"
    fig.savefig(out_png, dpi=600, bbox_inches="tight")
    plt.close(fig)

print("✅ Saved:", *[f"{prefix}_grouped_dfof_sorted_by_{tag}.png" for _, (_, tag) in sorters.items()], sep="\n- ")


C:\Users\suribear\AppData\Local\anaconda3\envs\neuro-group\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


✅ Saved:
- L472_f06_grouped_dfof_sorted_by_unsorted.png
- L472_f06_grouped_dfof_sorted_by_max_intensity.png
- L472_f06_grouped_dfof_sorted_by_pca.png
- L472_f06_grouped_dfof_sorted_by_kmeans.png
- L472_f06_grouped_dfof_sorted_by_hier.png
- L472_f06_grouped_dfof_sorted_by_corravg.png


##### Average rasterplots
Unsorted

Sorted by Pearson correlation

In [305]:
# averaged chunked_data shape: (neurons, time) sorted by stimulus type and by pearson correlation
# ---- 1) Extract chunks from the full dfof, averaged across repeats ----
chunked_data, trial_starts, move_starts, move_colors, stim_labels = st.extract_stimulus_chunks(
    deltaF_F=dfof,                 # (frames, neurons)
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    fps=fps_2p,
    stimuli_ordered=stimuli_ordered,
    window_pre=window_pre,
    window_post=window_post,
    average_across_repeats=True
)

if chunked_data is None or chunked_data.size == 0:
    raise ValueError("No stimulus chunks were extracted. Check exp_log, stimuli_ordered, and windows.")

# chunked_data shape: (neurons, time)  ✅

# ---- 2) Compute distances: 1 - Pearson correlation across the (averaged) chunked time series ----
dist = pdist(chunked_data, metric='correlation')  # rows = neurons, columns = time (averaged)
Z = linkage(dist, method='average')
sorted_indices = leaves_list(Z)

# ---- 3) Plot the correlation-sorted raster ----
fig, ax = plt.subplots(figsize=(9, 6))
im = plott.raster_with_stimuli(
    ax=ax,
    deltaF_F=chunked_data.T,    # (time, neurons)
    fps=fps_2p,
    fish_id=fish_id,
    neuron_order=sorted_indices,
    title_suffix='Average across trials • sorted by Pearson correlation',
    min=0.009,
    max=0.3
)


# 3) ➜ Overlay movement start lines using per-stim linestyle
for pos, stim_name in zip(move_starts, stim_labels):
    color = stimuli_colors.get(stim_name, 'black')
    ls    = stimuli_linestyles.get(stim_name, '-')   # default solid if not specified
    ax.axvline(pos / fps_2p, color=color, linestyle=ls, alpha=0.9, linewidth=1.5)

# 4) ➜ Legend that reflects both color and linestyle
legend_handles = []
for stim_name in stimuli_ordered:
    if stim_name in stimuli_colors:
        color = stimuli_colors[stim_name]
        ls    = stimuli_linestyles.get(stim_name, '-')  # same default
        (line,) = ax.plot([], [], color=color, linestyle=ls, label=stim_name, linewidth=1.5)
        legend_handles.append(line)

ax.legend(
    handles=legend_handles,
    title="Movement onset \n across stimuli",
    bbox_to_anchor=(1.2, 1),
    loc='upper left',
    borderaxespad=0,
    frameon=False
)

ax.set_xlabel("Chunks aligned to stimulus onset (s)")

# 5) Colorbar + layout ... [unchanged]
fig.colorbar(im, ax=ax, label="ΔF/F")
plt.subplots_adjust(right=0.85)
plt.show()



In [306]:
# Purpose.
# Generate experiment-level ΔF/F rasters aligned to stimulus onsets, averaged across repeats, and saved once per sorting strategy (unsorted, max intensity, PCA, KMeans, hierarchical, and correlation on averaged responses).

import os
os.environ["OMP_NUM_THREADS"] = "2"  # or "1"

# ----- pick a short prefix like "L433_f03" from "L433_f03_Exp_1_flickering"
prefix = "_".join(fish.split("_")[:2])

# ----- ensure output dir exists
plots_path.mkdir(parents=True, exist_ok=True)

# ===== 1) Build chunks (averaged across repeats) once =====
chunked_data, trial_starts, move_starts, move_colors, stim_labels = st.extract_stimulus_chunks(
    deltaF_F=dfof,                 # (frames, neurons)
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    fps=fps_2p,
    stimuli_ordered=stimuli_ordered,
    window_pre=window_pre,
    window_post=window_post,
    average_across_repeats=True
)
if chunked_data is None or chunked_data.size == 0:
    raise ValueError("No stimulus chunks were extracted. Check exp_log, stimuli_ordered, and windows.")
# chunked_data: (neurons, time)

# ===== 2) Precompute neuron orders for each sort mode (on full dfof unless noted) =====
deltaF_F = dfof  # (frames, neurons)

# (a) Max intensity (peak ΔF/F per neuron)
max_per_neuron   = np.nanmax(chunked_data.T, axis=0)         # (neurons,)
maxint_sorted_idx = np.argsort(-max_per_neuron)

# (b) PCA (PC1 desc) over neurons
scores    = PCA(n_components=3).fit_transform(chunked_data)  # (neurons, 3)
PCA_order = np.argsort(-scores[:, 0])

# (c) KMeans over neurons
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(chunked_data)
kmeans_sorted_idx = np.argsort(kmeans.labels_)  # group by label

# (d) Hierarchical (Ward) over neurons
Z_hier = linkage(chunked_data, method='ward')
clusters = fcluster(Z_hier, t=5, criterion='maxclust')
hier_sorted_idx = np.argsort(clusters)

# (e) Correlation sort on the **averaged chunks** (neurons x time)
dist = pdist(chunked_data, metric='correlation')
Z_corr = linkage(dist, method='average')
corravg_sorted_idx = leaves_list(Z_corr)

# Map from mode -> (index array or None, label suffix)
sorters = {
    "unsorted":        (None,                   "unsorted"),
    "max_intensity":   (maxint_sorted_idx,      "max_intensity"),
    "pca":             (PCA_order,              "pca"),
    "kmeans":          (kmeans_sorted_idx,      "kmeans"),
    "hier":            (hier_sorted_idx,        "hier"),
    "corravg":         (corravg_sorted_idx,     "corravg"),  # correlation on averaged chunks
}


def _validate_order(idx, n_neurons, name):
    if idx is None:
        return None
    if len(idx) != n_neurons:
        raise ValueError(f"{name} length ({len(idx)}) != number of neurons ({n_neurons}).")
    return idx

# ===== 3) Plot once per sort mode (use chunked_data for raster) =====
for mode, (idx, tag) in sorters.items():
    neuron_order = _validate_order(idx, chunked_data.shape[0], name=mode)

    fig, ax = plt.subplots(figsize=(9, 6))
    im = plott.raster_with_stimuli(
        ax=ax,
        deltaF_F=chunked_data.T,            # (time, neurons)
        fps=fps_2p,
        fish_id=fish_id,
        neuron_order=neuron_order,          # None = original order
        title_suffix=f"average across trials | sort={tag}",
        min=0.009,
        max=0.3
    )

    # Movement onset lines with per-stim color + linestyle
    for pos, stim_name in zip(move_starts, stim_labels):
        color = stimuli_colors.get(stim_name, 'black')
        ls    = stimuli_linestyles.get(stim_name, '-')  # default solid
        ax.axvline(pos / fps_2p, color=color, linestyle=ls, alpha=0.9, linewidth=1.8)

    # Legend (color + linestyle)
    legend_handles = []
    for stim_name in stimuli_ordered:
        if stim_name in stimuli_colors:
            color = stimuli_colors[stim_name]
            ls    = stimuli_linestyles.get(stim_name, '-')
            (line,) = ax.plot([], [], color=color, linestyle=ls, label=stim_name, linewidth=1.5)
            legend_handles.append(line)

    leg = ax.legend(
        handles=legend_handles, title="Movement onset\nacross stimuli",
        bbox_to_anchor=(1.2, 1), loc='upper left',
        borderaxespad=0, frameon=False
    )

    ax.set_xlabel("Chunks aligned to stimulus onset (s)")
    fig.colorbar(im, ax=ax, label="ΔF/F")
    plt.subplots_adjust(right=0.85)

    # Save
    out_png = plots_path / f"{prefix}_average_dfof_sorted_by_{tag}.png"
    fig.savefig(out_png, dpi=600, bbox_inches="tight")
    plt.close(fig)

print("✅ Saved:", *[f"{prefix}_average_dfof_sorted_by_{tag}.png" for _, (_, tag) in sorters.items()], sep="\n- ")


C:\Users\suribear\AppData\Local\anaconda3\envs\neuro-group\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


✅ Saved:
- L472_f06_average_dfof_sorted_by_unsorted.png
- L472_f06_average_dfof_sorted_by_max_intensity.png
- L472_f06_average_dfof_sorted_by_pca.png
- L472_f06_average_dfof_sorted_by_kmeans.png
- L472_f06_average_dfof_sorted_by_hier.png
- L472_f06_average_dfof_sorted_by_corravg.png


In [307]:
39830830# # Define neuron groups (in sorted neuron indices)
neuron_groups = {
    'C1 Left Bout': (140, 185),
    'C2 Right Bout': (85, 130),
    'C3 Continous': (65, 85)
}

# Colors for each group
group_colors = {
    'C1 Left Bout': 'green',
    'C2 Right Bout': 'firebrick',
    'C3 Continous': 'steelblue' #
}

# Extract stimulus-aligned chunks (already averages across repeats)
chunked_data, trial_starts, move_starts, move_colors, stim_labels = st.extract_stimulus_chunks(
    deltaF_F=fish_data['plane3']['deltaF_F'],
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    fps=fps_2p,
    stimuli_ordered=stimuli_ordered,
    window_pre=12,
    window_post=2,
    average_across_repeats=True
)

dist = pdist(chunked_data, metric='correlation')
Z = linkage(dist, method='average')
sorted_indices = leaves_list(Z)

selected_chuncks = 4*70

# Apply sorting
#sorted_data = chunked_data[sorted_indices, :selected_chuncks]
sorted_data = chunked_data[sorted_indices, :]

fig, ax = plt.subplots(figsize=(9, 6))
time_axis = np.arange(sorted_data.shape[1]) / fps_2p

for cluster_name, (start_idx, end_idx) in neuron_groups.items():
    cluster_data = sorted_data[start_idx:end_idx, :]  # select neurons in sorted space
    mean_trace = np.nanmean(cluster_data, axis=0)
    ax.plot(time_axis, mean_trace, color=group_colors[cluster_name], linewidth=2, label=cluster_name)


# Add vertical lines for movement onsets
for pos, color in zip(move_starts[:4], move_colors[:4]):
    ax.axvline(pos / fps_2p, color=color, alpha=0.9, linewidth=2.5, linestyle='--')

# Create legend
legend_handles = []
for stim_name, color in stimuli_colors.items():
    line, = ax.plot([], [], color=color, label=stim_name, linewidth=4)
    legend_handles.append(line)

ax.legend(handles=legend_handles, title="Movement onset \n across stimuli",
          bbox_to_anchor=(1.2, 1), loc='upper left', borderaxespad=0, frameon=False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_ylabel("Average ΔF/F")
ax.set_title(f"{fish_id} Plane 3 Average ΔF/F")
plt.subplots_adjust(right=0.85)
# fig.savefig(fig_paths / f'delta_average_plane3_allconditions.pdf', dpi=900)
# fig.savefig(fig_paths / f'delta_average_plane3_allconditions.png', dpi=1000)
plt.show()

NameError: name 'fish_data' is not defined

#### Trace ROIs back to anatomy

In [ ]:
stat = np.load(segmented_path / "plane3" / "stat.npy", allow_pickle=True) #load segmented ROIs
ops = np.load(segmented_path / "plane3" / "ops.npy", allow_pickle=True).item() #
mean_img = ops['meanImg']
roi_indices = fish_data['plane3']['ROIs_selected']

In [ ]:
chunked_data, trial_starts, move_starts, move_colors, stim_labels = st.extract_stimulus_chunks(
        deltaF_F=fish_data['plane3']['deltaF_F'],
        exp_log=exp_log,
        stimuli_durations=stimuli_durations,
        stimuli_colors=stimuli_colors,
        fps=fps_2p,
        stimuli_ordered=stimuli_ordered,
        window_pre=5,
        window_post=2,
        average_across_repeats=True)

dist = pdist(chunked_data, metric='correlation')
Z = linkage(dist, method='average')
sorted_indices = leaves_list(Z)

# Plot with sorted neurons
fig, ax = plt.subplots(figsize=(9, 6))
plott.raster_with_stimuli(
    ax=ax,
    deltaF_F=chunked_data.T,
    fps=fps_2p,
    plane_name='Plane 3',
    fish_id=fish_id,
    neuron_order=sorted_indices,
    title_suffix='Average across trials \n sorted by Pearson correlation'
)

for pos, color in zip(move_starts, move_colors):
    ax.axvline(pos / fps_2p, color=color, alpha=0.9, linewidth=1.5)

legend_handles = []
for stim_name, color in stimuli_colors.items():
    line, = ax.plot([], [], color=color, label=stim_name, linewidth=4)
    legend_handles.append(line)
ax.legend(handles=legend_handles, title="Movement onset \n across stimuli", bbox_to_anchor=(1.2, 1), loc='upper left', borderaxespad=0, frameon=False)
plt.subplots_adjust(right=0.85)
fig.savefig(fig_paths / f'raster_average_pearson_{plane_name}.pdf', dpi=900)
fig.savefig(fig_paths / f'raster_average_pearson_{plane_name}.png', dpi=1000)
plt.show()

In [ ]:
neuron_groups = {
    'C1 Left Bout': (140, 185),
    'C2 Right Bout': (85, 130),
    'C3 Continous': (65, 85)
}

# Colors for each group
group_colors = {
    'C1 Left Bout': 'green',
    'C2 Right Bout': 'firebrick',
    'C3 Continous': 'steelblue' #
}

In [ ]:
selected_neurons_sorted = np.arange(90, 135) #140, 185 left B,  130-190 right B
original_indices = roi_indices[sorted_indices[selected_neurons_sorted]]

fig, ax = plt.subplots(figsize=(6, 6))

p1, p99 = np.percentile(mean_img, (1, 99))
mean_img_clipped = np.clip(mean_img, p1, p99)
mean_img_norm = (mean_img_clipped - p1) / (p99 - p1)
plt.imshow(mean_img_norm, cmap='gray')

for i in original_indices:
    xcen = stat[i]['xpix'].mean()
    ycen = stat[i]['ypix'].mean()
    plt.plot(xcen, ycen, 'go', markersize=5)

plt.title(f"Plane 3 - ROIs {min(selected_neurons_sorted)}–{max(selected_neurons_sorted)} \n sorted by Pearson correlation")
ax.axis('off')
plt.tight_layout()
# fig.savefig(fig_paths / f'{plane_name}_ROIs_{min(selected_neurons_sorted)}–{max(selected_neurons_sorted)}_anatomy.pdf', dpi=900, bbox_inches='tight')
# fig.savefig(fig_paths / f'{plane_name}_ROIs_{min(selected_neurons_sorted)}–{max(selected_neurons_sorted)}_anatomy.png', dpi=1000, bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

# Normalize mean image for background
p1, p99 = np.percentile(mean_img, (1, 99))
mean_img_clipped = np.clip(mean_img, p1, p99)
mean_img_norm = (mean_img_clipped - p1) / (p99 - p1)
plt.imshow(mean_img_norm, cmap='gray')

# Loop over neuron groups
for cluster_name, (start_idx, end_idx) in neuron_groups.items():
    sorted_neurons = np.arange(start_idx, end_idx)
    original_indices = roi_indices[sorted_indices[sorted_neurons]]

    for i in original_indices:
        xcen = stat[i]['xpix'].mean()
        ycen = stat[i]['ypix'].mean()
        plt.plot(xcen, ycen, 'o', color=group_colors[cluster_name], markersize=5, label=cluster_name)

plt.title(f"Plane 3 - Anatomical location of selected ROIs")
ax.axis('off')
plt.tight_layout()
fig.savefig(fig_paths / f'plane3_ROIs_clusters_anatomy.pdf', dpi=900, bbox_inches='tight')
fig.savefig(fig_paths / f'plane3_ROIs_clusters_anatomy.png', dpi=1000, bbox_inches='tight')
plt.show()

Average